In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
sns.set()
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
df = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
store_info_data = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
test_data = pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')
# /kaggle/input/data-storm-4/

In [3]:
# df = df[df['quantity_sold'] > 0]

In [4]:
df = df[~df['customer_id'].str.contains('-')]

In [5]:
# # Create a mapping between item_price values and item_description values
# price_to_description = df.groupby('item_description')['item_price'].apply(lambda x: x.mode()[0]).to_dict()

# # Fill missing values in 'item_description' column using mapping
# df['item_description'].fillna(df['item_price'].map(price_to_description), inplace=True)

In [6]:
# df.dropna()
df.fillna(method='ffill',inplace=True)
df['quantity_sold'] = df.quantity_sold.abs()

df = df[df['quantity_sold'] > 0]

In [7]:
df[df.duplicated()].shape[0]
df.drop_duplicates(inplace=True)

In [8]:
# Group transactions by item description
grouped = df.groupby('item_description')

# Add a new column for item price with the mode value of each group
df['item_price'] = grouped['item_price'].transform(lambda x: x.mode()[0])

In [9]:
from datetime import datetime

def extract_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return date_obj.strftime("%Y-%m-%d")

df["transaction_date"] = df["transaction_date"].apply(extract_date)

# Convert the date column to a Pandas datetime object
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# Extract the year, month, day, and week of the year into separate columns
df["month"] = df["transaction_date"].dt.month
df["day"] = df["transaction_date"].dt.day
df["week"] = df["transaction_date"].dt.isocalendar().week

In [10]:
# Group the data by shop and customer, and count the number of transactions
transactions = df.groupby(["shop_id", "customer_id"])["invoice_id"].nunique()

# Count the number of customers who made more than one transaction at each shop
repeat_customers = transactions[transactions > 1].groupby("shop_id").count()

# Count the total number of unique customers who made a transaction at each shop
total_customers = transactions.groupby("shop_id").count()

total_invoices = transactions.groupby("shop_id").sum()

# Calculate the customer loyalty rate for each shop
loyalty_rate = repeat_customers / total_customers

# Reset the index of the loyalty_rate Series and rename the columns
loyalty_rate = loyalty_rate.reset_index()
loyalty_rate.columns = ["shop_id", "loyalty_rate"]

# Merge the loyalty rate with the original DataFrame on the "shop_id" column
df = pd.merge(df, loyalty_rate, on="shop_id")

In [11]:
total_customers = total_customers.reset_index()

total_customers.columns = ["shop_id", "no_customers"]


total_invoices = total_invoices.reset_index()

total_invoices.columns = ["shop_id", "no_invoices"]

number = pd.merge(total_customers, total_invoices, on="shop_id")

In [12]:
df = df.drop(["transaction_date","invoice_id"], axis= 1)

In [13]:
df['total_sales'] = df['item_price'] * df['quantity_sold']

In [14]:
df['week_group'] = pd.cut(df['week'], bins=[40, 45, 50], labels=['first5', 'last5'])

# df['week_group'] = pd.cut(df['week'], bins=[40, 42, 44, 46, 48, 50], labels=['week1', 'week2', 'week3', 'week4', 'week5'])

# Group by shop id and week group
grouped_df = df.groupby(['shop_id', 'week_group']).agg({'total_sales': 'sum'}).reset_index()

# Pivot the table to have first5 and last5 as columns
pivot_df = grouped_df.pivot(index='shop_id', columns='week_group', values='total_sales')

# Add a column to indicate whether a shop's total sales increased in the last 5 weeks compared to the first 5 weeks
pivot_df['increased'] = pivot_df['last5'] > pivot_df['first5']

# Reset the index and rename columns
result_df = pivot_df.reset_index()

result_df['percentage_increase'] = (result_df['last5'] - result_df['first5']) / result_df['first5'] * 100

# result_df.to_csv('result_df.csv', index=False)

# print(result_df)

result_df = result_df[['shop_id','increased','percentage_increase']]



In [15]:
result_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Calculate the mean of the percentage_increase column
mean_val = result_df['percentage_increase'].mean()

result_df.fillna(mean_val, inplace=True)

In [16]:
result_df.percentage_increase.unique()

array([34.11082597, 50.59327647, 23.65618528, 43.35897372, -4.76280032,
       36.09086141, 57.22847602, 26.17966024, 29.11555441, 12.34008986,
       10.30083789, 63.64955285, 29.993187  , 27.25620612, 43.58117782,
        3.09498069, 31.42949408, 20.14170173, 39.80284405, 26.01663171,
       45.50506283, 13.11777498, 24.37622169, 17.78773483, 38.06449401,
       39.89916125, 18.84759508, 32.69981173, 59.08179236, 11.2645225 ,
       14.18051172, 15.65402771, 16.27048859, 34.96729347, 14.56208231,
       16.66103128, 16.04388239, 25.03735138,  8.94203853, 24.26595144,
       22.8558252 , 20.08035658, 18.02964031, 26.59063074, 26.88998719,
       33.26888791, -4.86264863,  7.329907  , 47.68538406, 40.48901514,
        6.97326056, 33.05680205, 18.43471118,  5.55365179, 38.61742931,
       26.97986757, 21.81348342, 12.23759366, 25.58171781, 27.22868769,
       42.99452619, 41.12224595, 30.28576064, 13.16611661, 53.13697922,
       18.86046512, 27.18708781, 47.62150982, 42.19775691, 26.66

In [17]:
total_sales_3_week = df.loc[df.week.isin ([48,49,50])].groupby('shop_id')['total_sales'].sum()

In [18]:
# Group transactions by shop_id
shop_transactions = df.groupby('shop_id')

# Calculate summary statistics for each shop
shop_summary = shop_transactions.agg({
    'item_description': 'nunique', # number of unique items sold
    'quantity_sold': 'sum', # total quantity sold
    'total_sales': 'sum', # total sales
})

# Calculate loyalty rate mean for each shop
loyalty_rate_mean = df.groupby('shop_id')['loyalty_rate'].mean()

# Merge loyalty rate mean into shop_summary
shop_summary = shop_summary.merge(loyalty_rate_mean, on='shop_id')

# Group the data by shop and calculate the average item price
avg_item_price = df.groupby("shop_id")["item_price"].mean()

# Reset the index of the avg_item_price Series and rename the columns
avg_item_price = avg_item_price.reset_index()
avg_item_price.columns = ["shop_id", "avg_item_price"]

# Merge the avg_item_price with the original DataFrame on the "shop_id" column
shop_summary = shop_summary.merge(avg_item_price, on='shop_id')

In [19]:
shop_summary = shop_summary.merge(total_sales_3_week, on='shop_id')
shop_summary = shop_summary.rename(columns={'total_sales_x': 'total_sales', 'total_sales_y': 'total_sales_3_week'})

In [20]:
# Flatten column names
shop_summary.columns = [''.join(col).strip() for col in shop_summary.columns.values]

# Reset index to make shop_id a column again
shop_summary = shop_summary.reset_index()

In [21]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_3_week
0,0,SHOP001,35,3752,615770,0.228752,173.231511,247650
1,1,SHOP002,35,9040,1532215,0.233399,172.304377,611790
2,2,SHOP003,36,6548,1063720,0.217370,164.974710,366665
3,3,SHOP004,35,5969,971270,0.268561,165.500000,380155
4,4,SHOP005,36,7301,1245750,0.257195,172.540407,385900
...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3629,590830,0.235808,169.430380,202225
120,120,SHOP124,35,8506,1309570,0.247890,164.094645,448430
121,121,SHOP125,36,4079,526575,0.210407,138.314405,197455
122,122,SHOP126,33,6246,919030,0.219862,156.680532,296315


In [22]:
shop_summary = pd.merge(shop_summary, store_info_data, on='shop_id')

In [23]:
shop_summary = pd.merge(shop_summary, number, on='shop_id')

In [24]:
shop_summary = pd.merge(shop_summary, result_df, on='shop_id')

In [25]:
le = LabelEncoder()

shop_summary['increased'] = le.fit_transform(shop_summary['increased'])

In [26]:
shop_summary['sales_per_feet'] = shop_summary['total_sales'] / shop_summary['shop_area_sq_ft']
# shop_summary['sales_per_week'] = shop_summary['total_sales_3_week'] / 3
# shop_summary['sales_per_week_total'] = shop_summary['total_sales'] / 10

# shop_summary.to_csv('shop.csv', index=False)

In [27]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_3_week,shop_area_sq_ft,shop_profile,no_customers,no_invoices,increased,percentage_increase,sales_per_feet
0,0,SHOP001,35,3752,615770,0.228752,173.231511,247650,580,Low,1106,1542,1,34.110826,1061.672414
1,1,SHOP002,35,9040,1532215,0.233399,172.304377,611790,637,NaN,2545,3738,1,50.593276,2405.361068
2,2,SHOP003,36,6548,1063720,0.217370,164.974710,366665,810,High,2038,2858,1,23.656185,1313.234568
3,3,SHOP004,35,5969,971270,0.268561,165.500000,380155,516,Low,1724,2617,1,43.358974,1882.306202
4,4,SHOP005,36,7301,1245750,0.257195,172.540407,385900,617,Low,2189,3201,0,-4.762800,2019.043760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3629,590830,0.235808,169.430380,202225,735,Moderate,1145,1608,1,18.518382,803.850340
120,120,SHOP124,35,8506,1309570,0.247890,164.094645,448430,606,High,2251,3250,1,23.904049,2161.006601
121,121,SHOP125,36,4079,526575,0.210407,138.314405,197455,1028,High,884,1307,1,74.866240,512.232490
122,122,SHOP126,33,6246,919030,0.219862,156.680532,296315,328,Moderate,1742,2549,1,10.914891,2801.920732


In [28]:
from sklearn.metrics.pairwise import rbf_kernel


# Compute the age similarity feature using the RBF kernel
simil_617 = rbf_kernel(shop_summary[["shop_area_sq_ft"]], [[617]], gamma=0.1)

# Add the age similarity feature to the housing dataset
shop_summary['simil_617'] = np.squeeze(simil_617)

In [29]:
# Compute the similarity feature using the RBF kernel with center at 676
simil_676 = rbf_kernel(shop_summary[["shop_area_sq_ft"]], [[676]], gamma=0.1)

# Add the similarity feature to the shop_summary dataset
shop_summary['simil_676'] = np.squeeze(simil_676)


In [30]:
# shop_summary.to_csv('shop.csv', index=False)

shop_summary['log_item_price'] = np.log(shop_summary['avg_item_price'])

shop_summary['sqrt_avg_item_price'] = np.sqrt(shop_summary['avg_item_price'])

In [31]:
# import matplotlib.pyplot as plt
# shop_summary.hist(bins=100, figsize=(12,8))
# plt.show()

In [32]:
scaler = StandardScaler()

features = ['shop_area_sq_ft', "sqrt_avg_item_price","quantity_sold", "no_customers", "no_invoices",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase', "simil_617","simil_676"]

scaled_data = scaler.fit_transform(shop_summary[features])

shop_summary = shop_summary.drop(features, axis=1)

scaled_df = pd.DataFrame(scaled_data, columns=features)

shop_summary = pd.concat([shop_summary, scaled_df], axis=1)

In [33]:
# # define a function to cap the values in a column using the IQR method
# def cap_values(col):
#     q1 = col.quantile(0.25)
#     q3 = col.quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     col_capped = col.clip(lower_bound, upper_bound)
#     return col_capped

# # specify the columns to cap
# cols_to_cap = ['sales_per_feet', "avg_item_price",'loyalty_rate','total_sales','total_sales_3_week']

# # cap the values in each specified column
# for col in cols_to_cap:
#     shop_summary[col] = cap_values(shop_summary[col])

In [34]:
df_test = shop_summary[shop_summary['shop_profile'].isnull()]
df_test =  df_test.reset_index(drop=True)

In [35]:
df_train = shop_summary[shop_summary['shop_profile'].notnull()]
df_train =  df_train.reset_index(drop=True)

In [36]:
from scipy import stats

# define a function to remove outliers using Z-score method
def remove_outliers_zscore(df, col):
    # calculate the Z-score for each value in the column
    z = np.abs(stats.zscore(df[col]))
    # create a boolean mask where True = outlier
    mask = (z < 3)
    # return the dataframe with the outliers removed
    return df[mask]

# specify the columns to remove outliers
cols_to_remove_outliers = [ 'sales_per_feet','shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate', 'no_invoices','total_sales','total_sales_3_week','percentage_increase']

# remove outliers using Z-score method for each specified column
for col in cols_to_remove_outliers:
    df_train = remove_outliers_zscore(df_train, col)


In [37]:
# df_train.corr()

In [38]:
df_xx = df_train.drop('shop_profile', axis =1)
df_yy = df_train['shop_profile']

In [39]:
df_x, X_test, df_y, y_test = train_test_split(df_xx, df_yy, test_size=0.2, random_state=42)

In [40]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# Select k best features based on mutual information score
k = 5

f = df_x.drop('shop_id', axis=1)

selector = SelectKBest(mutual_info_classif, k=k)
selector.fit(f, df_y)

# Get indices of the selected features
idxs_selected = selector.get_support(indices=True)

# Get names of the selected features
selected_features = f.columns[idxs_selected].tolist()

# Print selected features
print(selected_features)


['quantity_sold', 'no_customers', 'no_invoices', 'total_sales', 'sales_per_feet']


In [41]:
# selected_features = ['shop_id','shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase',"simil_676"]
selected_features.append('shop_id')
df_x = df_x[selected_features]

df1_x = df_test.drop('shop_profile', axis =1)
df1_x = df1_x[selected_features]

In [42]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# corr = df_x.corr()

# fig, ax = plt.subplots(figsize=(10, 8))
# sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax)
# plt.title('Correlation Heatmap')
# plt.show()


In [43]:
X_train = df_x.drop('shop_id', axis =1 )
X_pred = df1_x.drop('shop_id', axis =1 )

In [44]:
X_test = X_test[selected_features]

X_test = X_test.drop('shop_id', axis =1 )


In [45]:
# lableEncoder = LabelEncoder()

# y_train = df_y

# y_train = lableEncoder.fit_transform(y_train)

In [46]:
# from sklearn.preprocessing import OrdinalEncoder

# encoder = OrdinalEncoder(categories=[["Low", "Moderate", "High"]])

# y_train = df_y

# encoder.fit(y_train)

# y_train = encoder.transform(y_train)

# print(y_train)


In [47]:
# y_train

In [48]:
encoding = {"Low": 1, "Moderate": 2, "High": 3}
y_train = list(map(encoding.get, df_y))

print(y_train)

[2, 2, 1, 1, 2, 3, 2, 3, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 3, 3, 1, 3, 3, 3, 1, 3, 1, 2, 2, 3, 3, 3, 3, 1, 3, 3, 2, 1, 2, 1, 2, 2, 1, 1, 3, 1, 3, 1, 1, 3, 1, 2, 3, 1, 1, 1, 2, 2, 3, 2, 2, 2, 3, 1, 3, 3, 1]


In [49]:
df_y

70    Moderate
17    Moderate
82         Low
83         Low
32    Moderate
        ...   
63        High
75         Low
16        High
96        High
54         Low
Name: shop_profile, Length: 76, dtype: object

In [50]:
# predictors = ['shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase',"simil_676"]
# for predictor in predictors:
#     covariance = np.cov(df_x[predictor], y_train)[0][1]
#     print(f'Covariance between {predictor} and Outcome: {covariance:.2f}')

In [51]:
# predictors = ['shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase',"simil_676"]
# for predictor in predictors:
#     correlation = np.corrcoef(df_x[predictor], y_train)[0][1]
#     print(f'Correlation between {predictor} and Outcome: {correlation:.2f}')

In [52]:
# import matplotlib.pyplot as plt
# df_train.hist(bins=100, figsize=(12,8))
# plt.show()

In [53]:
# X_train["increased"].value_counts()

In [54]:
X_train

,quantity_sold,no_customers,no_invoices,total_sales,sales_per_feet
70,-1.356385,-1.128613,-1.245715,-1.280011,-1.433682
17,0.966431,0.598369,0.779314,1.220954,0.913026
82,-0.594035,-0.677243,-0.507360,-0.256118,0.734429
83,-0.259405,0.038171,-0.060716,-0.036316,-0.128877
32,-0.576887,-0.728981,-0.605404,-0.486221,-0.160501
...,...,...,...,...,...
63,1.573959,1.749096,1.838429,1.589223,1.732090
75,0.536751,0.080988,0.601382,0.835082,0.285697
16,2.185896,1.192466,1.373629,1.746622,1.348647
96,0.766534,0.460996,0.418609,0.609484,0.466870


In [55]:
# X_test

In [56]:
# max_depths = [2*i for i in range(1,15)]
# min_samples_splits = [2,4,8]
# min_samples_leafs =  [1,2,4]
# n_estimator = 100
# best_score = 0

# for max_depth in max_depths:
#     for min_samples_split in min_samples_splits:
#         for min_samples_leaf in min_samples_leafs:
#             random_forest = RandomForestClassifier( max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

#             accuracy_list = cross_val_score(random_forest, X_train, y_train, cv=10)
#             accuracy = accuracy_list.mean()

#             if accuracy > best_score:
#                 best_score = accuracy
#                 best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
#                 print(f'Best hyperparameters: {best_params}')
#                 print(f'Accuracy: {best_score}')


# print(f'Best hyperparameters: {best_params}')
# print(f'Accuracy: {best_score}')

In [57]:
# print(X_test.info())

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV

# create a Logistic Regression model
mmodel = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 500, 1000, 5000],
}


# perform grid search to find the best hyperparameters
grid_search = GridSearchCV(mmodel, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# print the best hyperparameters and corresponding score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# make predictions on the test data using the best model
model = grid_search.best_estimator_
# y_pred = model.predict(X_test)

# train the model on the training data
# model.fit(X_train, y_train)

Best Hyperparameters: {'C': 10, 'max_iter': 100}
Best Score: 0.605


In [59]:
#  model = DecisionTreeClassifier()
# # model = RandomForestClassifier(max_depth= 8, min_samples_split= 2, min_samples_leaf= 1)
# # max_depth= 6, min_samples_split= 4, min_samples_leaf= 1
# # n_estimators = 100, max_depth=16, min_samples_split=2, min_samples_leaf=1

# model.fit(X_train, y_train)

In [60]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 70 to 54
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   quantity_sold   76 non-null     float64
 1   no_customers    76 non-null     float64
 2   no_invoices     76 non-null     float64
 3   total_sales     76 non-null     float64
 4   sales_per_feet  76 non-null     float64
dtypes: float64(5)
memory usage: 3.6 KB


In [61]:
# import xgboost as xgb
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# model = xgb.XGBClassifier()
# # colsample_bytree = 0.75, learning_rate= 0.1, max_depth= 3, n_estimators = 100, subsample= 0.75
# model.fit(X_train, y_train)

In [62]:
# make predictions on the test data
y_pred_test = model.predict(X_test)

reverse_encoding = {v: k for k, v in encoding.items()}

y_pred_test = [reverse_encoding[val] for val in y_pred_test]

# # evaluate the accuracy of the model
accuracy = accuracy_score(y_pred_test, y_test)
print("Accuracy:", accuracy)

y_pred = model.predict(X_pred)

Accuracy: 0.5263157894736842


In [63]:
scores = cross_val_score(model, X_train, y_train, cv=5)                                   
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.625      0.46666667 0.66666667 0.53333333 0.73333333]
Mean cross-validation score: 0.605


In [64]:
reverse_encoding = {v: k for k, v in encoding.items()}

y_pred = [reverse_encoding[val] for val in y_pred]

print(y_pred)

shop_profile = pd.DataFrame({'shop_profile': y_pred})

df_concat = pd.concat([df1_x, shop_profile], axis=1)

store_info = store_info_data.drop('shop_profile', axis =1)

df_concat = pd.merge(df_concat, store_info , on="shop_id" , how="left")

['High', 'Low', 'Low', 'Low', 'Moderate', 'Moderate', 'Low', 'Moderate', 'Moderate', 'Low', 'High', 'Low', 'Low', 'High', 'Low', 'Moderate', 'High', 'Moderate', 'High', 'Low', 'Moderate', 'Moderate', 'Low', 'High']


In [65]:
output = df_concat[['shop_id','shop_profile']]
print(output)

    shop_id shop_profile
0   SHOP002         High
1   SHOP019          Low
2   SHOP023          Low
3   SHOP024          Low
4   SHOP029     Moderate
5   SHOP030     Moderate
6   SHOP038          Low
7   SHOP044     Moderate
8   SHOP046     Moderate
9   SHOP050          Low
10  SHOP056         High
11  SHOP061          Low
12  SHOP070          Low
13  SHOP074         High
14  SHOP076          Low
15  SHOP080     Moderate
16  SHOP081         High
17  SHOP087     Moderate
18  SHOP092         High
19  SHOP096          Low
20  SHOP097     Moderate
21  SHOP107     Moderate
22  SHOP108          Low
23  SHOP114         High


In [66]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

# define cross-validation
cv = 5

# define F1-score with macro averaging as scoring metric
scoring = make_scorer(f1_score, average='macro')

# perform cross-validation
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)

# print mean F1-score and standard deviation
print('Mean F1-score:', scores.mean())
print('Standard deviation:', scores.std())

Mean F1-score: 0.5845125245125246
Standard deviation: 0.08145473193433564


In [67]:
# Save test predictions to file

output.to_csv('submission.csv', index=False)